In [1]:
# Import libraries
from pydantic import BaseModel, Field
from trustcall import create_extractor
from langchain_openai import ChatOpenAI
from codes.config.config import config


class Memory(BaseModel):
    content: str = Field(description="The main content of the memory. For example: User expressed interest in learning about French.")

# Inspect the tool calls made by Trustcall
class Spy:
    def __init__(self):
        self.called_tools = []

    def __call__(self, run):
        # Collect information about the tool calls made by the extractor.
        q = [run]
        while q:
            r = q.pop()
            if r.child_runs:
                q.extend(r.child_runs)
            if r.run_type == "chat_model":
                self.called_tools.append(
                    r.outputs["generations"][0][0]["message"]["kwargs"]["tool_calls"]
                )

# Initialize the spy
spy = Spy()

# Initialize the model
model = ChatOpenAI(api_key=config.openai_api_key.get_secret_value(), model="gpt-4o", temperature=0)

# Create the extractor
trustcall_extractor = create_extractor(
    model,
    tools=[Memory],
    tool_choice="Memory",
    enable_inserts=True,
)

# Add the spy as a listener
trustcall_extractor_see_all_tool_calls = trustcall_extractor.with_listeners(on_end=spy)

2025-09-29 20:38:20,661 - root - INFO - Configuration loaded for environment: development


In [2]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# Instruction
instruction = """Extract memories from the following conversation:"""

# Conversation
conversation = [HumanMessage(content="Hi, I'm peyman."),
                AIMessage(content="Nice to meet you, Peyman."),
                HumanMessage(content="I am an AI engineering student, In love with agent development.")]

# Invoke the extractor
result = trustcall_extractor.invoke({"messages": [SystemMessage(content=instruction)] + conversation})

2025-09-29 20:39:05,787 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'headers': {'X-Stainless-Raw-Response': 'true'}, 'files': None, 'idempotency_key': 'stainless-python-retry-338405cc-57ab-43a4-8daa-a534f369c9cd', 'json_data': {'messages': [{'content': 'Extract memories from the following conversation:', 'role': 'system'}, {'content': "Hi, I'm peyman.", 'role': 'user'}, {'content': 'Nice to meet you, Peyman.', 'role': 'assistant'}, {'content': 'I am an AI engineering student, In love with agent development.', 'role': 'user'}], 'model': 'gpt-4o', 'stream': False, 'temperature': 0.0, 'tool_choice': {'type': 'function', 'function': {'name': 'Memory'}}, 'tools': [{'type': 'function', 'function': {'name': 'Memory', 'description': None, 'parameters': {'properties': {'content': {'description': 'The main content of the memory. For example: User expressed interest in learning about French.', 'title': 'Content', 'type': 'string'}}, 'required': 

In [3]:
# Messages contain the tool calls
for m in result["messages"]:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  Memory (call_jf2ulpuJlOJkopJO4KSzdlVK)
 Call ID: call_jf2ulpuJlOJkopJO4KSzdlVK
  Args:
    content: Peyman is an AI engineering student who loves agent development.


In [4]:
# Metadata contains the tool call
for m in result["response_metadata"]:
    print(m)

{'id': 'call_jf2ulpuJlOJkopJO4KSzdlVK'}


In [5]:
# Update the conversation
updated_conversation = [AIMessage(content="That's great, did you do after?"),
                        HumanMessage(content="I went to Tartine and ate a croissant."),
                        AIMessage(content="What else is on your mind?"),
                        HumanMessage(content="I was thinking about my Japan, and going back this winter!"),]

# Update the instruction
system_msg = """Update existing memories and create new ones based on the following conversation:"""

# We'll save existing memories, giving them an ID, key (tool name), and value
tool_name = "Memory"
existing_memories = [(str(i), tool_name, memory.model_dump()) for i, memory in enumerate(result["responses"])] if result["responses"] else None
existing_memories

[('0',
  'Memory',
  {'content': 'Peyman is an AI engineering student who loves agent development.'})]

In [6]:
# Invoke the extractor with our updated conversation and existing memories
result = trustcall_extractor_see_all_tool_calls.invoke({"messages": updated_conversation,
                                                        "existing": existing_memories})

2025-09-29 20:40:01,500 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'headers': {'X-Stainless-Raw-Response': 'true'}, 'files': None, 'idempotency_key': 'stainless-python-retry-bb4287b9-02c2-432c-826a-5e8721e9e46f', 'json_data': {'messages': [{'content': 'Generate JSONPatches to update the existing schema instances. If you need to extract or insert *new* instances of the schemas, call the relevant function(s).\n<existing>\n<instance id=0 schema_type="Memory">\n{\'content\': \'Peyman is an AI engineering student who loves agent development.\'}\n</instance>\n</existing>\n', 'role': 'system'}, {'content': "That's great, did you do after?", 'role': 'assistant'}, {'content': 'I went to Tartine and ate a croissant.', 'role': 'user'}, {'content': 'What else is on your mind?', 'role': 'assistant'}, {'content': 'I was thinking about my Japan, and going back this winter!', 'role': 'user'}], 'model': 'gpt-4o', 'stream': False, 'temperature': 0.0, 

In [7]:
# Metadata contains the tool call
for m in result["response_metadata"]:
    print(m)


{'id': 'call_DsGEudb8dZZ6ALTujm0zBLJO', 'json_doc_id': '0'}
{'id': 'call_OoGspfcgrTbRt8CJBActfWGM'}


In [8]:
# Parsed responses
for m in result["responses"]:
    print(m)

content='Peyman is an AI engineering student who loves agent development. Recently, he went to Tartine and ate a croissant. He is also thinking about going back to Japan this winter.'
content='Peyman went to Tartine and ate a croissant. He is also thinking about going back to Japan this winter.'


In [9]:
spy.called_tools

[[{'name': 'PatchDoc',
   'args': {'json_doc_id': '0',
    'planned_edits': "1. Add a new memory about the user going to Tartine and eating a croissant. This will be added to the existing memory content.\n2. Add another memory about the user's thoughts on going back to Japan this winter.",
    'patches': [{'op': 'replace',
      'path': '/content',
      'value': 'Peyman is an AI engineering student who loves agent development. Recently, he went to Tartine and ate a croissant. He is also thinking about going back to Japan this winter.'}]},
   'id': 'call_DsGEudb8dZZ6ALTujm0zBLJO',
   'type': 'tool_call'},
  {'name': 'Memory',
   'args': {'content': 'Peyman went to Tartine and ate a croissant. He is also thinking about going back to Japan this winter.'},
   'id': 'call_OoGspfcgrTbRt8CJBActfWGM',
   'type': 'tool_call'}]]